## Requirements

In [ ]:
!pip install langchain==0.2.5 langchain-community==0.2.5 langchain-huggingface==0.0.3 chromadb==0.5.3 transformers==4.41.2

## Import Modules

In [5]:
import os
import getpass
import matplotlib.pyplot as plt

from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_huggingface import HuggingFaceEndpoint
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

## Show document

In [ ]:
with open('data.txt', 'r') as file:
    file_content = file.read()
print(file_content)

## Hugging-Face API Token

In [7]:
from google.colab import userdata
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')

# Prompt and Template

In [8]:
prompt_template = """Use the following pieces of context to answer the question at the end. If the context is not relevant,
      please don't answer the question using your own knowledge. Just say "I don't know".

      {context}

      Question: {question}
"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
    )

chain_type_kwargs = {"prompt": PROMPT}  # chain_type_kwargs will be pass on to Retreival QA chain

# Split Documents

In [ ]:
loader = TextLoader('data.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=20)    # chunk_overlap : size of overlaps between 2 chunks
texts = text_splitter.split_documents(documents)

# Embeddings

In [ ]:
# Use an embedding in hugging-face
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Generate Retriever

In [10]:
# create a Chroma vectorstore
db = Chroma.from_documents(texts, embeddings)

# search_type = similarity / mmr
retriever = db.as_retriever(search_type="similarity")

In [11]:
db.embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

# Model

In [17]:
# model using hugging-face
llm_llama = HuggingFaceEndpoint(
    repo_id = "NousResearch/Meta-Llama-3-8B",
    temperature = 0.5,
    max_new_tokens = 64,
    )

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [18]:
# QA retrieval
qa_llama = RetrievalQA.from_chain_type(
    llm=llm_llama,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs
    )

# Enter and run query

In [ ]:
question = input("your question: ")
result = qa_llama.invoke(question)
print(result)